In [1]:
# !pip install pandas

You are using pip version 9.0.3, however version 21.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import csv

df=pd.read_csv('../data/_fasr_data_cleaned_v2.csv')

# pre processing - scale ripple & efficiency
df['o1'] = df['o1']*100
df['o2'] = df['o2']/100
print(df)

            o1        o2        o3  f1  f2  f3  f4  f5  f6  f7  f8  f9  f10  \
0    18.664571  0.912755  7.320161   1   1   1  -1  -1  -1  -1  -1  -1    1   
1    15.820381  0.910424  7.482942   1   1   1  -1  -1  -1  -1  -1  -1    1   
2    21.132775  0.910279  7.586356   1   1   1  -1  -1  -1  -1  -1  -1    1   
3     8.800834  0.908270  7.762458   1   1   1  -1  -1  -1  -1  -1  -1    1   
4    10.378489  0.908696  7.833571   1  -1   1   1  -1  -1  -1  -1  -1    1   
..         ...       ...       ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...   
265  35.610421  0.900965  8.010373   1   1   1  -1  -1   1   1   1   1    1   
266  31.927450  0.902044  7.058374   1   1   1  -1  -1   1   1  -1   1    1   
267  19.580835  0.907175  8.317840   1   1   1  -1  -1   1   1   1  -1    1   
268  31.016359  0.907167  7.029862   1   1   1  -1  -1   1  -1  -1   1    1   
269  44.665721  0.907159  7.572992   1   1   1  -1  -1   1   1   1   1    1   

     f11  f12  f13  
0     -1   -1   -1  
1     -1 

In [3]:
"""
for ind in range(1, 13+1):
    print(f"factor{ind} list --> ",df[f'f{ind}'].tolist())

for ind in range(1, 3+1):
    print(f"output{ind} list --> ",df[f'o{ind}'].tolist())
"""

'\nfor ind in range(1, 13+1):\n    print(f"factor{ind} list --> ",df[f\'f{ind}\'].tolist())\n\nfor ind in range(1, 3+1):\n    print(f"output{ind} list --> ",df[f\'o{ind}\'].tolist())\n'

## Calculate factor effect Values

In [4]:
def get_factor_effect(factor_list, output_list, max_inp=1, min_inp=-1):  
    # same as 2factor2levels code
    factor_effect = 0
    mean_weightage = len(output_list)/2
    
    pos_effect = 0
    pos_counts = 0
    neg_effect = 0
    neg_counts = 0
    for fact_val, out_val in zip(factor_list, output_list):
        if fact_val == max_inp:    
            pos_effect += out_val
            pos_counts += 1
        
        elif fact_val == min_inp:
            neg_effect -= out_val
            neg_counts += 1
    
        else:
            print("something wrong in input")
    
    total_counts = pos_counts + neg_counts
    factor_effect = (pos_effect*neg_counts/total_counts) + (neg_effect*pos_counts/total_counts)
    # print('counts ', pos_counts, neg_counts)
    # print('effects ', pos_effect, neg_effect)
    
    return factor_effect/mean_weightage

In [5]:
_factor_names = [f'f{ind}' for ind in range(1, 13+1)]
_output_names = [f'o{ind}' for ind in range(1, 3+1)]

# rows are f1, f2, .. 13 rows, cols are o1, o2 .. 3 columns
output_df = pd.DataFrame(np.nan, columns=_output_names, index=_factor_names)

for factor_name, _row in output_df.iterrows():
    for output_name in _row.keys():
        _factor_effect = get_factor_effect(df[factor_name].tolist(), df[output_name].tolist())
        output_df.loc[factor_name][output_name] = _factor_effect


In [6]:
output_df['o_com'] = 0
for output_name in _output_names:
    output_df['o_com'] += abs(output_df[output_name])
output_df['o_com'] = output_df['o_com']/len(_output_names)

com_avg = output_df['o_com'].mean()
print(f'average of composite factor is {com_avg}\n\n')
print(output_df)

average of composite factor is 1.0514773574829432


           o1        o2        o3     o_com
f1   0.652043  0.000080 -0.021016  0.224380
f2   2.261710  0.000399 -0.074411  0.778840
f3   0.022290  0.000090 -0.021373  0.014584
f4  -4.294196  0.000221  0.074510  1.456309
f5  -1.590756  0.000552  0.064809  0.552039
f6   4.855269  0.000287 -0.071751  1.642436
f7   5.641002 -0.000047 -0.091588  1.910879
f8   5.541243 -0.000110 -0.089541  1.876965
f9   5.288502  0.000037 -0.098243  1.795594
f10 -1.945234 -0.000115 -0.031650  0.659000
f11  2.944511  0.000390  0.020520  0.988474
f12  2.495853  0.000430  0.063556  0.853280
f13  2.700374  0.000337  0.048570  0.916427


In [7]:
output_df.to_csv('../data/results_v2.csv')

In [8]:
df['f6'].value_counts()

 1    161
-1    109
Name: f6, dtype: int64

In [9]:
 get_factor_effect(df['f6'].tolist(), df['o1'].tolist())

4.855268863761316